In [ ]:
best_kg = 0

In [ ]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5

def MaxMinNorm10(df, df_scale):
    df_nom = ((df - df_scale.min())/(df_scale.max()-df_scale.min()))*10
    return df_nom

def SubmissionCsv(kg, model, name, predict):
    submit_file = 'c:\\house\\submission_' + name + '_' + str(int(kg/0.000001)) + '.csv'
    print submit_file
    submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": predict
    })

    submission.to_csv(submit_file, index=False)

    model_file = 'c:\\house\\' + name + '_' + str(int(kg/0.000001)) + '.pkl'
    print model_file
    joblib.dump(model, model_file) 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

%matplotlib inline
#panda is a read/write tool

train_file = "c:\\house\\train.csv"
test_file = "c:\\house\\test.csv"

# compare with the almost data
almost_perfect_file = "c:\\house\\almost_perfect.csv"
almostPref = pd.read_csv(almost_perfect_file, header = 0)
alpSlp = almostPref['SalePrice']

In [ ]:
hsData = pd.read_csv(train_file, header = 0)

#hsData = hsData[hsData['GrLivArea'] < 5000]
#hsData = hsData[hsData['GrLivArea'] > 400]
hsData['Qualified_SF'] = hsData['GrLivArea'] - hsData['LowQualFinSF']
hsData['Qualified_SF'] = np.log(hsData['Qualified_SF'])
hsData = hsData.drop(['1stFlrSF','2ndFlrSF','LowQualFinSF'], axis=1)

hsData = hsData[hsData['LotArea'] < 100000]
hsData['LotArea'] = np.log(hsData['LotArea'])

#hsData = hsData[hsData['TotalBsmtSF'] < 3000]
#hsData = hsData[hsData['GarageArea'] < 1200]

hsData['Porch_SF'] = hsData['OpenPorchSF'] + hsData['EnclosedPorch'] + hsData['3SsnPorch'] + hsData['ScreenPorch']
hsData = hsData.drop(['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'], axis=1)

hsData['Total_Bath'] = hsData['FullBath'] + hsData['BsmtFullBath'] + hsData['HalfBath']*0.5 + hsData['BsmtHalfBath']*0.5
hsData = hsData.drop(['FullBath','BsmtFullBath','HalfBath','BsmtHalfBath'], axis=1)

#drop negative correlation
hsData = hsData.drop(['KitchenAbvGr','YrSold','MoSold','MiscVal'], axis=1)

hsData['LotFrontage'] = hsData['LotFrontage'].fillna(0)
hsData['MasVnrType'] = hsData['MasVnrType'].fillna('None')
hsData['MasVnrArea'] = hsData['MasVnrArea'].fillna(0)
hsData['GarageYrBlt'] = hsData['GarageYrBlt'].fillna(0)
hsData['BsmtQual'] = hsData['BsmtQual'].fillna('NA')
hsData['BsmtCond'] = hsData['BsmtCond'].fillna('NA')
hsData['BsmtExposure'] = hsData['BsmtExposure'].fillna('NA')
hsData['FireplaceQu'] = hsData['FireplaceQu'].fillna('NA')
hsData['GarageType'] = hsData['GarageType'].fillna('NA')
hsData['GarageFinish'] = hsData['GarageFinish'].fillna('NA')
hsData['GarageQual'] = hsData['GarageQual'].fillna('NA')
hsData['GarageCond'] = hsData['GarageCond'].fillna('NA')
hsData['PoolQC'] = hsData['PoolQC'].fillna('NA')
#hsData['Fence'] = hsData['Fence'].fillna('NA')

hsData['MSSubClass'].replace([180,30,45,190,50,90,85,40,160,70,20,75,80,120,60], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], inplace=True)
hsData['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [1,3,5,7,9], inplace=True)
#hsData['Street'].replace(['Grvl','Pave'], [1,2], inplace=True)

hsData['MasVnrType'].replace(['BrkCmn','None','BrkFace','Stone'], [1,3,7,9], inplace=True)

hsData['LotShape'].replace(['Reg','IR1','IR3','IR2'], [1,3,5,7], inplace=True)
#hsData['LandContour'].replace(['Bnk','Lvl','Low','HLS'], [1,5,7,9], inplace=True)
hsData['LandContour'].replace(['Bnk','Lvl','Low','HLS'], [139400,162900,190000,222250], inplace=True)

#hsData['Neighborhood'].replace(['MeadowV','IDOTRR','BrDale','OldTown','Edwards','BrkSide','Sawyer','Blueste','SWISU','NAmes','NPkVill','Mitchel','SawyerW','Gilbert','NWAmes','Blmngtn','CollgCr','ClearCr','Crawfor','Veenker','Somerst','Timber','StoneBr','NoRidge','NridgHt'], \
#                                [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25], inplace=True)
hsData['Neighborhood'].replace(['MeadowV','IDOTRR','BrDale','OldTown','Edwards','BrkSide','Sawyer','Blueste','SWISU','NAmes','NPkVill','Mitchel','SawyerW','Gilbert','NWAmes','Blmngtn','CollgCr','ClearCr','Crawfor','Veenker','Somerst','Timber','StoneBr','NoRidge','NridgHt'], \
                                [88000,103000,106000,119000,121750,124300,135000,137500,139500,140000,146000,153500,179900,181000,182900,191000,197200,200250,200624,218000,225500,228475,278000,301500,315000], inplace=True)

hsData['Condition1'].replace(['Artery','Feedr','RRAe','Norm','RRAn','RRNe','PosN','PosA','RRNn'], [1,2,3,4,5,6,7,8,9], inplace=True)
hsData['Condition2'].replace(['RRNn','Artery','Feedr','RRAn','Norm','RRAe','PosN','PosA'], [1,2,3,4,5,6,7,8], inplace=True)

hsData['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [1,2,3,5,7], inplace=True)
hsData['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [1,2,3,4,5,6,7,8], inplace=True)

hsData['ExterQual'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)
hsData['ExterCond'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)

hsData['BsmtQual'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsData['BsmtCond'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsData['BsmtExposure'].replace(['NA','No','Mn','Av','Gd'], [0,1,3,5,7], inplace=True)

hsData['HeatingQC'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)
hsData['CentralAir'] = hsData['CentralAir'].apply(lambda air: 1 if air == 'Y' else 0)

hsData['KitchenQual'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)
hsData['FireplaceQu'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)

hsData['GarageType'].replace(['NA','CarPort','Detchd','Basment','2Types','Attchd','BuiltIn'], [0,1,3,5,7,9,11], inplace=True)
hsData['GarageFinish'].replace(['NA','Unf','RFn','Fin'], [0,1,5,9], inplace=True)
hsData['GarageQual'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsData['GarageCond'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsData['PavedDrive'].replace(['N','P','Y'], [0,1,3], inplace=True)

hsData['Has_Pool'] = hsData['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
hsData = hsData.drop(['PoolArea'], axis=1)
hsData['PoolQC'].replace(['NA','Fa','TA','Gd','Ex'], [0,3,5,7,9], inplace=True)

#hsData['Has_Fence'] = hsData['Fence'].apply(lambda x: 0 if pd.isnull(x) else 1)
#hsData['Fence'].replace(['NA','MnWw','GdWo','MnPrv','GdPrv'], [0,3,3,3,5], inplace=True)

hsData['SaleType'].replace(['Oth','ConLI','COD','ConLD','ConLw','WD','CWD','New','Con'], [1,2,3,4,5,6,7,8,9], inplace=True)
hsData['SaleCondition'].replace(['AdjLand','Abnorml','Family','Alloca','Normal','Partial'], [1,3,5,7,9,11], inplace=True)

#hsData.info()

In [ ]:
num_train_features = hsData.select_dtypes(include=[np.number])
#num_train_features = preprocessing.scale(num_train_features)

corr = num_train_features.corr()
#print (corr['SalePrice'].sort_values(ascending=False)[:70], '\n')
num_train_features = num_train_features.drop(['Id', 'SalePrice'], axis=1)
#print num_train_features.iloc[:5]

null_columns = num_train_features.columns[num_train_features.isnull().any()]
num_train_features[null_columns].isnull().sum()

In [ ]:
num_train_features['YearBuilt'] = MaxMinNorm10(hsData['YearBuilt'], hsData['YearBuilt'])
num_train_features['YearRemodAdd'] = MaxMinNorm10(hsData['YearRemodAdd'], hsData['YearRemodAdd'])
num_train_features['TotalBsmtSF'] = MaxMinNorm10(hsData['TotalBsmtSF'], hsData['TotalBsmtSF'])
num_train_features['Porch_SF'] = MaxMinNorm10(hsData['Porch_SF'], hsData['Porch_SF'])
num_train_features['GarageArea'] = MaxMinNorm10(hsData['GarageArea'], hsData['GarageArea'])
num_train_features['GarageYrBlt'] = MaxMinNorm10(hsData['GarageYrBlt'], hsData['GarageYrBlt'])
num_train_features['MasVnrArea'] = MaxMinNorm10(hsData['MasVnrArea'], hsData['MasVnrArea'])
num_train_features['Neighborhood'] = MaxMinNorm10(hsData['Neighborhood'], hsData['Neighborhood'])
num_train_features['LandContour'] = MaxMinNorm10(hsData['LandContour'], hsData['LandContour'])

#rint num_train_features['Qualified_SF'].iloc[:5]
#rint num_train_features['LotArea'].iloc[:5]
#int num_train_features['TotalBsmtSF'].iloc[:5]
#rint num_train_features['Porch_SF'].iloc[:5]
#rint num_train_features['GarageArea'].iloc[:5]
#print num_train_features['LandContour'].iloc[35:45]

In [ ]:
#num_train_features.skew()

In [ ]:
hsTest = pd.read_csv(test_file, header = 0)

hsTest['Qualified_SF'] = hsTest['GrLivArea'] - hsTest['LowQualFinSF']
hsTest['Qualified_SF'] = np.log(hsTest['Qualified_SF'])
hsTest = hsTest.drop(['1stFlrSF','2ndFlrSF','LowQualFinSF'], axis=1)

hsTest['LotArea'] = np.log(hsTest['LotArea'])

hsTest['Porch_SF'] = hsTest['OpenPorchSF'] + hsTest['EnclosedPorch'] + hsTest['3SsnPorch'] + hsTest['ScreenPorch']
hsTest = hsTest.drop(['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'], axis=1)

hsTest['TotalBsmtSF'] = hsTest['TotalBsmtSF'].fillna(0)
hsTest['BsmtFullBath'] = hsTest['BsmtFullBath'].fillna(0)
hsTest['BsmtHalfBath'] = hsTest['BsmtHalfBath'].fillna(0)
hsTest['Total_Bath'] = hsTest['FullBath'] + hsTest['BsmtFullBath'] + hsTest['HalfBath']*0.5 + hsTest['BsmtHalfBath']*0.5
hsTest = hsTest.drop(['FullBath','BsmtFullBath','HalfBath','BsmtHalfBath'], axis=1)

hsTest = hsTest.drop(['KitchenAbvGr','YrSold','MoSold','MiscVal'], axis=1)

hsTest['MSZoning'] = hsTest['MSZoning'].fillna('RM')
hsTest['LotFrontage'] = hsTest['LotFrontage'].fillna(0)
hsTest['MasVnrType'] = hsTest['MasVnrType'].fillna('None')
hsTest['MasVnrArea'] = hsTest['MasVnrArea'].fillna(0)
hsTest['GarageYrBlt'] = hsTest['GarageYrBlt'].fillna(0)
hsTest['GarageCars'] = hsTest['GarageCars'].fillna(0)
hsTest['GarageArea'] = hsTest['GarageArea'].fillna(0)
hsTest['BsmtFinSF1'] = hsTest['BsmtFinSF1'].fillna(0)
hsTest['BsmtFinSF2'] = hsTest['BsmtFinSF2'].fillna(0)
hsTest['BsmtUnfSF'] = hsTest['BsmtUnfSF'].fillna(0)
hsTest['BsmtQual'] = hsTest['BsmtQual'].fillna('NA')
hsTest['BsmtCond'] = hsTest['BsmtCond'].fillna('NA')
hsTest['BsmtExposure'] = hsTest['BsmtExposure'].fillna('NA')
hsTest['KitchenQual'] = hsTest['KitchenQual'].fillna('Po')
hsTest['FireplaceQu'] = hsTest['FireplaceQu'].fillna('NA')
hsTest['GarageType'] = hsTest['GarageType'].fillna('NA')
hsTest['GarageFinish'] = hsTest['GarageFinish'].fillna('NA')
hsTest['GarageQual'] = hsTest['GarageQual'].fillna('NA')
hsTest['GarageCond'] = hsTest['GarageCond'].fillna('NA')
hsTest['PoolQC'] = hsTest['PoolQC'].fillna('NA')
#hsTest['Fence'] = hsTest['Fence'].fillna('NA')


hsTest['MSSubClass'].replace([180,30,45,190,50,90,85,40,160,70,20,75,80,120,60], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], inplace=True)
hsTest['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [1,3,5,7,9], inplace=True)
#hsTest['Street'].replace(['Grvl','Pave'], [1,2], inplace=True)

hsTest['MasVnrType'].replace(['BrkCmn','None','BrkFace','Stone'], [1,3,7,9], inplace=True)

hsTest['LotShape'].replace(['Reg','IR1','IR3','IR2'], [1,3,5,7], inplace=True)
#hsTest['LandContour'].replace(['Bnk','Lvl','Low','HLS'], [1,5,7,9], inplace=True)
hsTest['LandContour'].replace(['Bnk','Lvl','Low','HLS'], [139400,162900,190000,222250], inplace=True)

#hsTest['Neighborhood'].replace(['MeadowV','IDOTRR','BrDale','OldTown','Edwards','BrkSide','Sawyer','Blueste','SWISU','NAmes','NPkVill','Mitchel','SawyerW','Gilbert','NWAmes','Blmngtn','CollgCr','ClearCr','Crawfor','Veenker','Somerst','Timber','StoneBr','NoRidge','NridgHt'], \
#                                [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25], inplace=True)
hsTest['Neighborhood'].replace(['MeadowV','IDOTRR','BrDale','OldTown','Edwards','BrkSide','Sawyer','Blueste','SWISU','NAmes','NPkVill','Mitchel','SawyerW','Gilbert','NWAmes','Blmngtn','CollgCr','ClearCr','Crawfor','Veenker','Somerst','Timber','StoneBr','NoRidge','NridgHt'], \
                                [88000,103000,106000,119000,121750,124300,135000,137500,139500,140000,146000,153500,179900,181000,182900,191000,197200,200250,200624,218000,225500,228475,278000,301500,315000], inplace=True)

hsTest['Condition1'].replace(['Artery','Feedr','RRAe','Norm','RRAn','RRNe','PosN','PosA','RRNn'], [1,2,3,4,5,6,7,8,9], inplace=True)
hsTest['Condition2'].replace(['RRNn','Artery','Feedr','RRAn','Norm','RRAe','PosN','PosA'], [1,2,3,4,5,6,7,8], inplace=True)

hsTest['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [1,2,3,5,7], inplace=True)
hsTest['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [1,2,3,4,5,6,7,8], inplace=True)

hsTest['ExterQual'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)
hsTest['ExterCond'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)

hsTest['BsmtQual'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsTest['BsmtCond'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsTest['BsmtExposure'].replace(['NA','No','Mn','Av','Gd'], [0,1,3,5,7], inplace=True)

hsTest['HeatingQC'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)
hsTest['CentralAir'] = hsTest['CentralAir'].apply(lambda air: 1 if air == 'Y' else 0)

hsTest['KitchenQual'].replace(['Po','Fa','TA','Gd','Ex'], [1,3,5,7,9], inplace=True)
hsTest['FireplaceQu'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)

hsTest['GarageType'].replace(['NA','CarPort','Detchd','Basment','2Types','Attchd','BuiltIn'], [0,1,3,5,7,9,11], inplace=True)
hsTest['GarageFinish'].replace(['NA','Unf','RFn','Fin'], [0,1,5,9], inplace=True)
hsTest['GarageQual'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsTest['GarageCond'].replace(['NA','Po','Fa','TA','Gd','Ex'], [0,1,3,5,7,9], inplace=True)
hsTest['PavedDrive'].replace(['N','P','Y'], [0,1,3], inplace=True)

hsTest['Has_Pool'] = hsTest['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
hsTest = hsTest.drop(['PoolArea'], axis=1)
hsTest['PoolQC'].replace(['NA','Fa','TA','Gd','Ex'], [0,3,5,7,9], inplace=True)

#hsTest['Has_Fence'] = hsTest['Fence'].apply(lambda x: 0 if pd.isnull(x) else 1)
#hsTest['Fence'].replace(['NA','MnWw','GdWo','MnPrv','GdPrv'], [0,3,3,3,5], inplace=True)

hsTest['SaleType'] = hsTest['SaleType'].fillna('WD')
hsTest['SaleType'].replace(['Oth','ConLI','COD','ConLD','ConLw','WD','CWD','New','Con'], [1,2,3,4,5,6,7,8,9], inplace=True)
hsTest['SaleCondition'].replace(['AdjLand','Abnorml','Family','Alloca','Normal','Partial'], [1,3,5,7,9,11], inplace=True)

#hsTest.info()

In [ ]:
num_test_features = hsTest.select_dtypes(include=[np.number])
num_test_features = num_test_features.drop(['Id'], axis=1)

null_columns=num_test_features.columns[num_test_features.isnull().any()]
num_test_features[null_columns].isnull().sum()

In [ ]:
num_test_features['YearBuilt'] = MaxMinNorm10(hsTest['YearBuilt'], hsData['YearBuilt'])
num_test_features['YearRemodAdd'] = MaxMinNorm10(hsTest['YearRemodAdd'], hsData['YearRemodAdd'])
num_test_features['TotalBsmtSF'] = MaxMinNorm10(hsTest['TotalBsmtSF'], hsData['TotalBsmtSF'])
num_test_features['Porch_SF'] = MaxMinNorm10(hsTest['Porch_SF'], hsData['Porch_SF'])
num_test_features['GarageArea'] = MaxMinNorm10(hsTest['GarageArea'], hsData['GarageArea'])
num_test_features['GarageYrBlt'] = MaxMinNorm10(hsTest['GarageYrBlt'], hsData['GarageYrBlt'])
num_test_features['MasVnrArea'] = MaxMinNorm10(hsTest['MasVnrArea'], hsData['MasVnrArea'])
num_test_features['Neighborhood'] = MaxMinNorm10(hsTest['Neighborhood'], hsData['Neighborhood'])
num_test_features['LandContour'] = MaxMinNorm10(hsTest['LandContour'], hsData['LandContour'])

#print num_test_features['Qualified_SF'].iloc[:5]
#print num_test_features['LotArea'].iloc[:5]
#print num_test_features['TotalBsmtSF'].iloc[:5]
#print num_test_features['Porch_SF'].iloc[:5]
#print num_test_features['GarageArea'].iloc[:5]
#print num_test_features['LandContour'].iloc[:5]

In [ ]:
#LassoCV Regressor
from sklearn.linear_model import LassoCV
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib


X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

lscv = LassoCV(eps=10**-7, n_alphas=75)
lscv.fit(X_train, y_train)

X_test_predict = lscv.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(lscv)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Lscv_test_predict = lscv.predict(num_test_features)
Lscv_test_predict = np.exp(Lscv_test_predict)
'''
plt.grid(True)
plt.scatter(X_test_predict, y_test, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('LassoCV Regressor')
plt.show()
'''

print("Predict %s : %s\n" % (hsTest['Id'][1089], Lscv_test_predict[1089]))
plt.grid(True)
plt.scatter(Lscv_test_predict, alpSlp, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('LassoCV Regressor')
plt.show()

kg = rmsle(alpSlp, Lscv_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

if best_kg == 0:
    best_kg = kg
    print best_kg
    print("Best Kaggle: %s\n" % best_kg)
elif best_kg > kg:
    best_kg = kg
    print("Best Kaggle: %s\n" % best_kg)
    
if  kg < 0.12 and Lscv_test_predict[1089] < 570000:
    SubmissionCsv(kg, lscv, 'lscv', Lscv_test_predict)

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestRegressor(n_estimators = 170, max_depth=5)
rf.fit(X_train, y_train)

X_test_predict = rf.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(rf)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Rf_test_predict = rf.predict(num_test_features)
Rf_test_predict = np.exp(Rf_test_predict)

print("Predict %s : %s\n" % (hsTest['Id'][1089], Rf_test_predict[1089]))
plt.grid(True)
plt.scatter(Rf_test_predict, alpSlp, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Random Forest Regressor')
plt.show()

kg = rmsle(alpSlp, Rf_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

if best_kg == 0:
    best_kg = kg
    print best_kg
    print("Best Kaggle: %s\n" % best_kg)
elif best_kg > kg:
    best_kg = kg
    print("Best Kaggle: %s\n" % best_kg)
    
if  kg < 0.15 and Rf_test_predict[1089] < 300000:
    SubmissionCsv(kg, rf, 'rf', Rf_test_predict)
    

In [ ]:
# Bagging Regressor
from sklearn.ensemble import BaggingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

bag = BaggingRegressor(n_estimators=150)
bag.fit(X_train, y_train)

X_test_predict = bag.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(bag)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Bag_test_predict = bag.predict(num_test_features)
Bag_test_predict = np.exp(Bag_test_predict)

print("Predict %s : %s\n" % (hsTest['Id'][1089], Bag_test_predict[1089]))
plt.grid(True)
plt.scatter(Bag_test_predict, alpSlp, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Bagging Regressor')
plt.show()

kg = rmsle(alpSlp, Bag_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

if best_kg == 0:
    best_kg = kg
    print best_kg
    print("Best Kaggle: %s\n" % best_kg)
elif best_kg > kg:
    best_kg = kg
    print("Best Kaggle: %s\n" % best_kg)
    
if  kg < 0.15 and Bag_test_predict[1089] < 400000:
    SubmissionCsv(kg, bag, 'bag', Bag_test_predict)
    

In [ ]:
# Bagging Regressor
from xgboost import XGBRegressor
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib

X = num_train_features
y = np.log(hsData['SalePrice'])

X_train, X_test, y_train, y_test = train_test_split(X, y)

xg = XGBRegressor(colsample_bytree=0.2,
                 gamma=0.0,
                 learning_rate=0.01,
                 max_depth=4,
                 min_child_weight=1.5,
                 n_estimators=7200,                                                                  
                 reg_alpha=0.9,
                 reg_lambda=0.6,
                 subsample=0.2,
                 seed=42,
                 silent=1)
xg.fit(X_train, y_train)

X_test_predict = xg.predict(X_test)

X_test_predict = np.exp(X_test_predict)
y_test = np.exp(y_test)

print(xg)
print("RMSLE: %s\n" % rmsle(y_test, X_test_predict))

Xg_test_predict = xg.predict(num_test_features)
Xg_test_predict = np.exp(Xg_test_predict)

print("Predict %s : %s\n" % (hsTest['Id'][1089], Xg_test_predict[1089]))
plt.grid(True)
plt.scatter(Xg_test_predict, alpSlp, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Bagging Regressor')
plt.show()

kg = rmsle(alpSlp, Xg_test_predict) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

if best_kg == 0:
    best_kg = kg
    print best_kg
    print("Best Kaggle: %s\n" % best_kg)
elif best_kg > kg:
    best_kg = kg
    print("Best Kaggle: %s\n" % best_kg)
    
if  kg < 0.15 and Xg_test_predict[1089] < 400000:
    SubmissionCsv(kg, xg, 'xg', Xg_test_predict)

In [ ]:
print("Best Kaggle: %s\n" % best_kg)

In [ ]:
lscv_file = "c:\\house\\submission_lscv_117851.csv"
rf_file = "c:\\house\\submission_rf_149600.csv"
bag_file = "c:\\house\\submission_bag_134776.csv"
xg_file = "c:\\house\\submission_xg_111492.csv"

lscv_res = pd.read_csv(lscv_file, header = 0)
rf_res = pd.read_csv(rf_file, header = 0)
bag_res = pd.read_csv(bag_file, header = 0)
xg_res = pd.read_csv(xg_file, header = 0)

ensemble_res = (lscv_res['SalePrice'] + xg_res['SalePrice'] + bag_res['SalePrice'])/3

print("Predict %s : %s\n" % (hsTest['Id'][1089], ensemble_res[1089]))
plt.grid(True)
plt.scatter(ensemble_res, alpSlp, alpha=.25,color='b') #alpha helps to show overlapping data
plt.plot([50000,500000],[50000,500000],'r--')
plt.xlabel('Predicted Price')
plt.ylabel('Actual Price')
plt.title('Bagging Regressor')
plt.show()

kg = rmsle(alpSlp, ensemble_res) - 0.0109768754926667
print("Kaggle Possible: %s\n" % kg)

submit_file = 'c:\\house\\submission_ensemble_' + str(int(kg/0.000001)) + '.csv'
print submit_file
submission = pd.DataFrame({
        "Id": hsTest["Id"],
        "SalePrice": ensemble_res
    })

submission.to_csv(submit_file, index=False)